<a href="https://colab.research.google.com/github/bhushanmandava/Gradiant-Boosting-Algos-Classification/blob/main/xgboost_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGBoost Classifier

## Part 1 - Data Preprocessing

### Importing the dataset

In [113]:
import numpy as np
import pandas as pd


In [114]:
dataset =pd.read_csv('churn_modelling.csv')

### Checking missing data

In [115]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


In [116]:
dataset.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Handling categorical variables

CustomerId and Surname columns

In [117]:
dataset=dataset.drop(['CustomerId','Surname'],axis=1)

In [118]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Geography column

In [119]:
dataset['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [120]:
geography_dummies = pd.get_dummies(dataset['Geography'], drop_first=True)
# geography_dummies.apply(lambda x:0 if x==False else 1)
geography_dummies = geography_dummies.applymap(lambda x: 0 if x == False else 1)
geography_dummies.head()

<ipython-input-120-c246ffd8da37>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  geography_dummies = geography_dummies.applymap(lambda x: 0 if x == False else 1)


,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1


In [121]:
dataset = pd.concat([geography_dummies,dataset],axis=1)
dataset.head()

,Germany,Spain,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,0,1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,0,0,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,0,0,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,0,1,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [122]:
dataset = dataset.loc[:,~dataset.columns.duplicated()]
#getting every row and columns exept the ones that are duplicated

In [123]:
dataset.head()

,Germany,Spain,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,0,1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,0,0,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,0,0,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,0,1,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [124]:
dataset=dataset.drop(['Geography'],axis=1)

Gender column

In [125]:
dataset['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [126]:
dataset["Gender"]=dataset["Gender"].apply(lambda x: 0 if x=='Female' else 1)

In [127]:
dataset.head()

,Germany,Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,0,619,0,42,2,0.00,1,1,1,101348.88,1
1,0,1,608,0,41,1,83807.86,1,0,1,112542.58,0
2,0,0,502,0,42,8,159660.80,3,1,0,113931.57,1
3,0,0,699,0,39,1,0.00,2,0,0,93826.63,0
4,0,1,850,0,43,2,125510.82,1,1,1,79084.10,0


### Creating the Training Set and the Test Set

Getting the inputs and output

In [128]:
x=dataset.iloc[:,:-1].values
y=dataset.iloc[:,-1].values

Getting the Training Set and the Test Set

In [129]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

## Part 2 - Building and training the model

### Building the model

In [130]:
import xgboost as xgb
model=xgb.XGBClassifier(max_depth = 4, learning_rate = 0.1, n_estimators = 100)


### Training the model

In [131]:
model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

### Inference

In [132]:
y_pred=model.predict(x_test)

### Predicting the result of a single observation



Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [133]:

single_pred=model.predict([[0,0,600,1,40,3,60000,2,1,1,50000]])
if(single_pred[0]==0):
    print("Customer will not leave the bank")
else:
    print("Customer will leave the bank")

Customer will not leave the bank


Therefore, our model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "0, 0" in the first two columns. That's because of course the predict method expects the dummy values of the Geography variable.

## Part 3: Evaluating the model

### Making the Confusion Matrix

In [134]:
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
# y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)


[[1526   69]
 [ 191  214]]


### Accuracy

In [135]:
accuracy_score(y_test, y_pred)

0.87

### f1 score

In [136]:
f1_score(y_test, y_pred)

0.622093023255814

### k-Fold Cross Validation

In [137]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model, X = x, y = y, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Devation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 86.43 %
Standard Devation: 0.77 %
